In [27]:
import json
import os
import requests

from typing import Any

customer_id = "2906470669"
corpus_id = "2"
api_key = os.getenv("VECTARA_API_KEY") or "VECTARA_API_KEY"
url = f"https://api.vectara.io/v1/upload?c={customer_id}&o={corpus_id}"

post_headers = {
    "x-api-key": api_key,
    "customer-id": customer_id
}

In [37]:
directory = "./markdown"
markdown_files = []
for root, dirs, files in os.walk(directory):
    for file_name in files:
        # Check if the file is a markdown file
        if file_name.endswith('.md'):
            file_path = os.path.join(root, file_name)

            # Read the file content
            with open(file_path, "r", encoding="utf-8") as file:
                file_lines = file.readlines()
                file_name_stubs = file_name.split(".")[0].split("_")
                meta_data = dict(
                    title=file_lines[0][1:].strip(),
                    url="",
                    category=file_lines[1][12:].strip(),
                    category_slug=file_name_stubs[0],
                    slug=file_name_stubs[1],
                    file_name=file_name,
                    file_path=file_path
                )
                # Append the file path and content to the markdown_files list
                markdown_files.append((file_path, meta_data))

In [ ]:
responses = []
for mdf in markdown_files:
    files: Any = {
        "file": (mdf[1]["file_name"], open(mdf[0], "rb")),
        "doc_metadata": (None, json.dumps(mdf[1])),
    }
    response = requests.post(url, files=files, verify=True, headers=post_headers)
    responses.append(response)

In [38]:
errors = [r for r in responses if r.status_code != 200]

In [39]:
errors

[]

In [40]:
responses[0].text

'{"response":{\n  "status": {\n  },\n  "quotaConsumed": {\n    "numChars": "47389",\n    "numMetadataChars": "16648"\n  }\n}}'